# 必要なライブラリのインポート

In [ ]:
import sys
import os

parent_dir = os.path.abspath("..")
sys.path.append(parent_dir)

import ccxt
from src.data_fetcher.gmo import GmoFetcher
import joblib
import tempfile
import datetime
import numpy as np
import pandas as pd
from pathlib import Path

pd.set_option("display.max_columns", None)

# パラメータ管理

In [ ]:
# 日付設定
today_utc = pd.Timestamp.utcnow().normalize()
today_str = today_utc.strftime("%Y%m%d")
one_year_ago_utc = today_utc - pd.DateOffset(years=1)

# 取得する通貨ペア
market = "SOL"

# データ取得の時間間隔（分）
interval_min = 1

# GMO Fetcherの初期化
cache_dir = os.path.join(tempfile.gettempdir(), "gmo_fetcher_cache")
memory = joblib.Memory(cache_dir, verbose=0)
fetcher = GmoFetcher(memory=memory)

# 保存先の設定
data_folder = Path("../data/ohlcv")
data_folder.mkdir(parents=True, exist_ok=True)
ohlcv_file_path = data_folder / f'df_ohlcv_{market}_{interval_min}min_{today_str}.parquet'

## データを用意

GMOコインのAPIから取引データを取得し、OHLCV形式に変換します。

In [ ]:
df = fetcher.fetch_ohlcv(
    market='BTC_JPY',
    interval_sec=15 * 60,
)

In [ ]:
df

## ohlcvをファイルに保存

In [ ]:
df.to_parquet(ohlcv_file_path)